# "COVID-19 plots"
> "Interactive plots with daily updates"

- author: bbernst
- toc: true
- branch: master
- badges: true
- comments: true
- categories: [healthcare, covid]
- image: images/covid_plots/covid_mask.png
- hide: false
- search_exclude: true

I wanted to make a simple COVID-19 post to accomplish two main goals:
1. Easily see the specific plots I'm interested in
1. Update these plots every day, automatically

Enter [fastpages](https://github.com/fastai/fastpages) and [github actions](https://github.com/features/actions). I'd been meaning to set up a blog with fastpages for a while and thought this would be a good reason to try it out. The documentation was great and I was able to set up automatic updates by using https://covid19dashboards.com/ as a guide.

The data for this post comes from https://covidtracking.com/, which updates by 5pm each day, and I have my dashboard update at 6:30pm. For each geography -- so far that's the US, New York, and Non Tri-State -- I look at daily positive cases, positive rate, and deaths. Be sure to hover around on each of the plots, they have helpful interactive tooltips to make reading the data more intuitive.

In [1]:
#collapse
from typing import Callable, Literal
import pandas as pd
import altair as alt

In [2]:
#hide_input
current_utc = pd.Timestamp.now(tz='America/New_York').strftime('%Y-%m-%d %I:%M:%S %p')
print(f"Last updated at (ET): {current_utc}")

Last updated at (ET): 2021-01-08 04:33:44 AM


# Data

In [3]:
df = pd.read_csv('https://covidtracking.com/api/states/daily.csv')

/opt/hostedtoolcache/Python/3.8.6/x64/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Helpers

In [4]:
#collapse
def create_df(df: pd.DataFrame, 
              filter_f: Callable, 
              state_name: str):
    out_df = (
     df
     .assign(date = lambda d: pd.to_datetime(d['date'], format='%Y%m%d'))
     .loc[filter_f, :]
     .groupby('date')
     .apply(lambda d: pd.Series(dict(
         totalTestResultsIncrease = d['totalTestResultsIncrease'].sum(),
         positiveIncrease = d['positiveIncrease'].sum(),
         death = d['death'].sum(),
         deathIncrease = d['deathIncrease'].sum()
     )))
     .reset_index()
     .assign(positiveRate = lambda d: round(d['positiveIncrease'] / d['totalTestResultsIncrease'], 3))
     .sort_values('date', ascending=True)
     .assign(state = state_name)
     .reset_index(drop=True)
    )
    
    return out_df

In [5]:
#collapse
def create_plot(df: pd.DataFrame, 
                column: Literal['totalTestResultsIncrease', 
                                'positiveIncrease', 
                                'death',
                                'deathIncrease', 
                                'positiveRate'],
                period: int = 7*8):
    df = (
     df
     .assign(sma7d = lambda d: d[column].rolling(window=7).mean().round(3))
     .loc[lambda d: d['date'] >= pd.to_datetime('today') - period * pd.Timedelta('1 days'), :]
     .reset_index(drop=True)
    )
    the_state = df['state'][0]
    label = alt.selection_single(
        encodings=['x'], on='mouseover', nearest=True, empty='none')
    
    base = alt.Chart().mark_point(size=75).encode(
        x='date:T',
        y=f'{column}:Q',
        tooltip=['date', f'{column}:Q', 'sma7d:Q']
    )

    sma = alt.Chart().mark_line(color='purple').encode(
        x='date:T',
        y='sma7d:Q'
    )

    layers = alt.layer(
        base + sma,

        # add a rule mark to serve as a guide line
        alt.Chart().mark_rule(color='#aaa').encode(
            x='date:T'
        ).transform_filter(label),

        # add circle marks for selected time points, hide unselected points
        base.mark_circle().encode(
            opacity=alt.condition(label, alt.value(1), alt.value(0))
        ).add_selection(label),

        # add white stroked text to provide a legible background for labels
        base.mark_text(align='left', dx=5, dy=-5, stroke='white', strokeWidth=2).encode(
            text=f'{column}:Q'
        ).transform_filter(label),

        # add text labels
        base.mark_text(align='left', dx=5, dy=-5).encode(
            text=f'{column}:Q'
        ).transform_filter(label),
        
        data=df
    ).properties(
        title=f'{the_state}: {column}',
        width=500,
        height=400
    )
    
    return layers.interactive()

# Plots

## US

In [6]:
us = create_df(df, lambda d: d.index > 0, 'US')

In [7]:
create_plot(us, 'positiveIncrease')

alt.LayerChart(...)

In [8]:
create_plot(us, 'positiveRate')

alt.LayerChart(...)

In [9]:
create_plot(us, 'deathIncrease')

alt.LayerChart(...)

## NY

In [10]:
ny = create_df(df, (lambda d: d['state'] == 'NY'), 'New York')

In [11]:
create_plot(ny, 'positiveIncrease')

alt.LayerChart(...)

In [12]:
create_plot(ny, 'positiveRate')

alt.LayerChart(...)

In [13]:
create_plot(ny, 'deathIncrease')

alt.LayerChart(...)

## Non Tri-State

In [14]:
non_tristate = create_df(df, lambda d: ~d['state'].isin(['CT','NJ','NY']), 'Non Tri-State')

In [15]:
create_plot(non_tristate, 'positiveIncrease')

alt.LayerChart(...)

In [16]:
create_plot(non_tristate, 'positiveRate')

alt.LayerChart(...)

In [17]:
create_plot(non_tristate, 'deathIncrease')

alt.LayerChart(...)